In [1]:
NUM_NEIGHBORS_LIST = [10, 10] 

for k, hopk_num in enumerate(NUM_NEIGHBORS_LIST):
        print("k: {} hopk_num: {}".format(k, hopk_num))

k: 0 hopk_num: 10
k: 1 hopk_num: 10


In [8]:
import numpy as np

edges=np.array([[1, 2], [1, 3], [2, 3]])

print(edges[:, 0])

adj = dict(zip(edges[:, 0], edges[:, 1]))

print(adj)

[1 1 2]
{1: 3, 2: 3}
